In [1]:
import cv2
import numpy as np

from VoltricDP.Preprocess import ColorMask


class TransformPos(object):

    def __init__(self):
        None

vid = cv2.VideoCapture(1)

pygame 2.1.2 (SDL 2.0.18, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
# Webcam & Canvas Dim
width = 1280
height = 720

# Define Color range in HSV
color = np.uint8([[[128, 0, 128]]])  # in rgb
hsvGreen = cv2.cvtColor(color, cv2.COLOR_RGB2HSV)
lower = hsvGreen[0][0][0] - 30, 127, 127
upper = hsvGreen[0][0][0] + 30, 255, 255

noise = (20, 20)
turd = (20, 20)

# Webcam/Any Input Feed

vid.set(cv2.CAP_PROP_FRAME_WIDTH, width)
vid.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

# Function for frame
get_frame = lambda: cv2.flip(vid.read()[1], 1)

# Initialize Preprocessor for Input
CM = ColorMask(lower, upper, kernel_noise=noise, kernel_turd=turd)
sample = get_frame()
sample = np.zeros_like(sample)

background = cv2.cvtColor(sample, cv2.COLOR_BGR2GRAY)
background = np.full_like(background, 255)

sample = cv2.line(sample, (0, height - 11), (width - 1, height - 11),
                  color=[128, 0, 128], thickness=20)

corners = np.float32([[0, 0], [0, height - 1], [width - 1, 0], [width - 1, height - 1]])
field = np.float32([[200, 200], [200, height - 201], [width - 201, 200], [width - 201, height - 201]])

h = cv2.getPerspectiveTransform(field, corners)

cv2.namedWindow("window", cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty("window", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

while True:

    cv2.waitKey(1000 // 30)
    frame = get_frame()
    # backup = frame.copy()
    mask = CM.get_mask(frame)
    res = cv2.bitwise_and(background, background, mask=mask)

    cols = res.shape[1] // 30
    horizontalStructure = cv2.getStructuringElement(cv2.MORPH_RECT, (cols, 1))
    horizontal = cv2.erode(res, horizontalStructure)
    horizontal = cv2.dilate(horizontal, horizontalStructure)

    contours, centers = CM.get_cc(horizontal)
    # print(contours)
    centers = np.float32(np.array(centers))
    centers = np.expand_dims(centers, axis=1)
    # pts = np.float32(np.array([[[1920, 1280]]]))
    canvas = sample.copy()

    if len(centers) > 0:
        abs_axis = cv2.perspectiveTransform(centers, h)
        for pp in abs_axis:
            print(tuple(pp[0].astype(int)))
            canvas = cv2.circle(canvas, tuple(pp[0].astype(int)), radius=30, color=(0, 0, 255), thickness=-1)
    # cv2.drawContours(canvas,contours, -1, (0, 255, 0), 10)
    else:
        # print('empty')
        None
    cv2.imshow('vid', frame)
    cv2.imshow('mask', mask)
    cv2.imshow('window', canvas)